In [ ]:
!unzip -O 'cp949' "2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803.zip"

Archive:  2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803.zip
   creating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/01_제공데이터/
  inflating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/01_제공데이터/2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803.xlsx  
   creating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/02_평가데이터/
  inflating: 2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/02_평가데이터/2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_평가데이터_210803.xlsx  


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta

In [ ]:
## 공통
base='2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/'
train_folder='01_제공데이터/'

data=pd.read_excel(base+train_folder+'2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803.xlsx')

cols=['홍수사상번호','연','월','일','시간','유입량']

for i in range(1,7):
    prefix=f'데이터집단{i}_'
    cols.append(prefix+'유역평균강수')
    cols.append(prefix+'강우(A지역)')
    cols.append(prefix+'강우(B지역)')
    cols.append(prefix+'강우(C지역)')
    cols.append(prefix+'강우(D지역)')
    cols.append(prefix+'수위(E지역)')
    cols.append(prefix+'수위(B지역)')

data=data.drop(0).reset_index(drop=True)
data.columns=cols

for idx,col in enumerate(cols):
    if idx<5:
        data[col]=data[col].astype(int)
    else:
        data[col]=data[col].astype(float)

In [ ]:
## reproducibility
import os
import random
import tensorflow as tf

seed=71

def seedEvery(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seedEvery(seed)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split    
        
from datetime import datetime,timedelta

data['time']=data.apply(lambda row:datetime(year=int(row['연']),month=int(row['월']),day=int(row['일'])),axis=1)
data['time']=data.apply(lambda row:row['time']+timedelta(hours=row['시간']),axis=1)

In [ ]:
## 빅콘테스트 수정사항
data.loc[2199,'홍수사상번호']=20
data=data.sort_values(by=['홍수사상번호','time']).reset_index(drop=True)

test=data.iloc[2891:]

submit=test[['홍수사상번호','연','월','일','시간']]
submit['유입량']=0

data=data.drop(['연','월','일','시간'],axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
## 유입량 < 10000
idx2=data[data['홍수사상번호']==1].loc[data['유입량']>=10000].index
data=data.drop(idx2,axis=0).reset_index(drop=True)

In [ ]:
## lag 변수
lag_3=data.set_index('time').groupby('홍수사상번호')[[col for col in data.columns if '강우' in col]].shift(3).reset_index().fillna(method='bfill')

lag_3.columns=['time']+['lag3_'+col for col in lag_3.columns[1:]]

data=pd.merge(data,lag_3,on='time',how='left')
data=data.drop(['time'],axis=1)

In [ ]:
## 정규화
## 나중에 예측값 내고. (pred+mean)*std 하기.

from sklearn.preprocessing import StandardScaler

for col in data.columns[1:]:
    if col=='유입량':
        mean=data[col].mean()
        std=data[col].std()
        data[col]=(data[col]-mean)/std
        continue
    ss=StandardScaler()
    data[col]=ss.fit_transform(data[col].values.reshape(-1,1))
data.head()

,홍수사상번호,유입량,데이터집단1_유역평균강수,데이터집단1_강우(A지역),데이터집단1_강우(B지역),데이터집단1_강우(C지역),데이터집단1_강우(D지역),데이터집단1_수위(E지역),데이터집단1_수위(B지역),데이터집단2_유역평균강수,데이터집단2_강우(A지역),데이터집단2_강우(B지역),데이터집단2_강우(C지역),데이터집단2_강우(D지역),데이터집단2_수위(E지역),데이터집단2_수위(B지역),데이터집단3_유역평균강수,데이터집단3_강우(A지역),데이터집단3_강우(B지역),데이터집단3_강우(C지역),데이터집단3_강우(D지역),데이터집단3_수위(E지역),데이터집단3_수위(B지역),데이터집단4_유역평균강수,데이터집단4_강우(A지역),데이터집단4_강우(B지역),데이터집단4_강우(C지역),데이터집단4_강우(D지역),데이터집단4_수위(E지역),데이터집단4_수위(B지역),데이터집단5_유역평균강수,데이터집단5_강우(A지역),데이터집단5_강우(B지역),데이터집단5_강우(C지역),데이터집단5_강우(D지역),데이터집단5_수위(E지역),데이터집단5_수위(B지역),데이터집단6_유역평균강수,데이터집단6_강우(A지역),데이터집단6_강우(B지역),데이터집단6_강우(C지역),데이터집단6_강우(D지역),데이터집단6_수위(E지역),데이터집단6_수위(B지역),lag3_데이터집단1_강우(A지역),lag3_데이터집단1_강우(B지역),lag3_데이터집단1_강우(C지역),lag3_데이터집단1_강우(D지역),lag3_데이터집단2_강우(A지역),lag3_데이터집단2_강우(B지역),lag3_데이터집단2_강우(C지역),lag3_데이터집단2_강우(D지역),lag3_데이터집단3_강우(A지역),lag3_데이터집단3_강우(B지역),lag3_데이터집단3_강우(C지역),lag3_데이터집단3_강우(D지역),lag3_데이터집단4_강우(A지역),lag3_데이터집단4_강우(B지역),lag3_데이터집단4_강우(C지역),lag3_데이터집단4_강우(D지역),lag3_데이터집단5_강우(A지역),lag3_데이터집단5_강우(B지역),lag3_데이터집단5_강우(C지역),lag3_데이터집단5_강우(D지역),lag3_데이터집단6_강우(A지역),lag3_데이터집단6_강우(B지역),lag3_데이터집단6_강우(C지역),lag3_데이터집단6_강우(D지역)
0,1,-0.841637,-1.056500,-0.983152,-0.934386,-0.712692,-0.809595,-0.886461,-1.472248,-1.069682,-1.012258,-0.927517,-0.710539,-0.809595,-0.886461,-1.489093,-1.069682,-1.012258,-0.873637,-0.717134,-0.817334,-0.886461,-1.495082,-1.079422,-1.026774,-0.873637,-0.830143,-0.843068,-0.886461,-1.430240,-1.065010,-1.026774,-0.873637,-0.830143,-0.843068,-0.886461,-1.439777,-1.017052,-1.026774,-0.881985,-0.695770,-0.822802,-0.886461,-1.454281,-0.958907,-0.909183,-0.693856,-0.787988,-0.98739,-0.902843,-0.692329,-0.787988,-0.98739,-0.851463,-0.699475,-0.795502,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.859493,-0.681334,-0.800985
1,1,-0.825271,-1.058261,-0.983152,-0.917497,-0.712692,-0.809595,-0.891035,-1.473277,-1.067914,-1.012258,-0.910481,-0.710539,-0.809595,-0.891035,-1.487720,-1.067914,-1.012258,-0.855492,-0.717134,-0.817334,-0.891035,-1.493707,-1.063454,-1.026774,-0.855492,-0.790995,-0.806740,-0.891035,-1.431417,-1.048773,-1.026774,-0.855492,-0.790995,-0.806740,-0.891035,-1.441754,-0.999933,-1.026774,-0.864008,-0.649239,-0.785648,-0.891035,-1.455927,-0.958907,-0.909183,-0.693856,-0.787988,-0.98739,-0.902843,-0.692329,-0.787988,-0.98739,-0.851463,-0.699475,-0.795502,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.859493,-0.681334,-0.800985
2,1,-0.805016,-1.056500,-0.983152,-0.900608,-0.712692,-0.809595,-0.891035,-1.474306,-1.052005,-1.012258,-0.893445,-0.710539,-0.809595,-0.891035,-1.486347,-1.052005,-1.012258,-0.837347,-0.694442,-0.817334,-0.891035,-1.492332,-1.047485,-1.026774,-0.837347,-0.790995,-0.788576,-0.891035,-1.432594,-1.032537,-1.026774,-0.837347,-0.790995,-0.788576,-0.891035,-1.443730,-0.982813,-1.026774,-0.846030,-0.649239,-0.767072,-0.891035,-1.457572,-0.958907,-0.909183,-0.693856,-0.787988,-0.98739,-0.902843,-0.692329,-0.787988,-0.98739,-0.851463,-0.699475,-0.795502,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.859493,-0.681334,-0.800985
3,1,-0.774821,-1.040648,-0.983152,-0.883718,-0.712692,-0.809595,-0.891035,-1.474306,-1.036096,-1.012258,-0.876409,-0.687945,-0.809595,-0.891035,-1.484973,-1.036096,-1.012258,-0.819202,-0.649059,-0.780800,-0.891035,-1.490957,-0.992483,-0.999278,-0.819202,-0.693122,-0.734085,-0.891035,-1.433771,-0.976610,-0.999278,-0.819202,-0.693122,-0.734085,-0.891035,-1.446366,-0.923847,-0.999278,-0.828053,-0.532912,-0.711342,-0.891035,-1.458395,-0.958907,-0.909183,-0.693856,-0.787988,-0.98739,-0.902843,-0.692329,-0.787988,-0.98739,-0.851463,-0.699475,-0.795502,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.851463,-0.809129,-0.820640,-1.001584,-0.859493,-0.681334,-0.800985
4,1,-0.726655,-1.024797,-0.983152,-0.849939,-0.690495,-0.773121,-0.891035,-1.474306,-0.981299,-0.984565,-0.842336,-0.642758,-0.773121,-0.891035,-1.483600,-0.981299,-0.984565,-0.782912,-0.649059,-0

In [ ]:
def window_dataset(df,label=None,window_size=20):
    if label is not None:
        window_list=[]
        label_list=[]
        for i in range(df.shape[0]-window_size):
            window_list.append(np.array(df[i:i+window_size]))
            label_list.append(np.array(label.iloc[i+window_size]))

        return np.array(window_list),np.array(label_list)
    else:
        window_list=[]
        for i in range(df.shape[0]-window_size):
            window_list.append(np.array(df[i:i+window_size]))
        return np.array(window_list)

In [ ]:
data[data['홍수사상번호']==26].index[0],data[data['홍수사상번호']==25].index[0],data[data['홍수사상번호']==24].index[0]

(2865, 2762, 2668)

In [ ]:
from sklearn.decomposition import PCA

lpca=PCA(n_components=30)
data_pca=lpca.fit_transform(data.iloc[:,2:])

label=data['유입량']

In [ ]:
train_x=data_pca[:2668]
valid_x=data_pca[:2762]

In [ ]:
test=data_pca

train_feature=train_x
train_label=label[:2668]
valid_feature=valid_x
valid_label=label[:2762]

train_feature, train_label=window_dataset(df=train_feature,label=train_label)
valid_feature, valid_label=window_dataset(df=valid_feature,label=valid_label)
test_feature=window_dataset(df=test)

print(f"train_feature.shape : {train_feature.shape}, train_label.shape : {train_label.shape}")
print(f"valid_feature.shape : {valid_feature.shape}, valid_label.shape : {valid_label.shape}")
print(f"test_feature.shape : {test_feature.shape}")

train_feature.shape : (2648, 20, 30), train_label.shape : (2648,)
valid_feature.shape : (2742, 20, 30), valid_label.shape : (2742,)
test_feature.shape : (3005, 20, 30)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import *

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def lstm(df,n_lstm):
    model=keras.models.Sequential([
        LSTM(n_lstm,input_shape=[df.shape[1],df.shape[2]],return_sequences=False,activation='tanh'),
        Dense(1)
        ])
    model.compile(loss=root_mean_squared_error,
                  optimizer='adam',
                  metrics =[tf.keras.metrics.RootMeanSquaredError()])
    return model

params={'df':train_feature,
        'n_lstm': 60}

lstm_model=lstm(**params)

lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60)                21840     
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 21,901
Trainable params: 21,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#model training
checkpoint_cb = keras.callbacks.ModelCheckpoint('lstm_model_pca.h5',
                                                monitor='val_root_mean_squared_error',
                                                save_best_only=True)
earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = lstm_model.fit(train_feature, train_label, epochs=1000, 
       validation_data = (valid_feature, valid_label), verbose=1, batch_size=16,
              callbacks = [checkpoint_cb, earlystopping_cb])

Epoch 1/1000
166/166 [==============================] - 5s 18ms/step - loss: 0.4061 - root_mean_squared_error: 0.4918 - val_loss: 0.1866 - val_root_mean_squared_error: 0.2530
Epoch 2/1000
166/166 [==============================] - 2s 14ms/step - loss: 0.1973 - root_mean_squared_error: 0.2152 - val_loss: 0.1371 - val_root_mean_squared_error: 0.1845
Epoch 3/1000
166/166 [==============================] - 2s 14ms/step - loss: 0.1588 - root_mean_squared_error: 0.1754 - val_loss: 0.1195 - val_root_mean_squared_error: 0.1594
Epoch 4/1000
166/166 [==============================] - 2s 14ms/step - loss: 0.1375 - root_mean_squared_error: 0.1519 - val_loss: 0.1110 - val_root_mean_squared_error: 0.1478
Epoch 5/1000
166/166 [==============================] - 2s 14ms/step - loss: 0.1218 - root_mean_squared_error: 0.1366 - val_loss: 0.0931 - val_root_mean_squared_error: 0.1270
Epoch 6/1000
166/166 [==============================] - 2s 14ms/step - loss: 0.1090 - root_mean_squared_error: 0.1228 - val_l

In [ ]:
lstm_model.load_weights('/content/lstm_model_pca.h5')
pred = lstm_model.predict(valid_feature)

## mean 더하고 std 곱하기
scaled_pred=(pred*std)+mean
scaled_pred=[x[0] if x[0]>0 else 0 for x in scaled_pred]
scaled_test_label=(valid_label*std)+mean

from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(scaled_test_label,scaled_pred)**0.5
rmse 

151.5710491983212

In [ ]:
pred=lstm_model.predict(test_feature)
pred=(pred*std)+mean
pred=[x[0] if x[0]>0 else 0 for x in pred][-160:]

submit_lstm=submit.copy()
submit_lstm['유입량']=pred
submit_lstm.head()

,홍수사상번호,연,월,일,시간,유입량
2891,26,2018,7,1,6,427.081055
2892,26,2018,7,1,7,148.893921
2893,26,2018,7,1,8,182.546387
2894,26,2018,7,1,9,267.955078
2895,26,2018,7,1,10,356.224609
